In [411]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from itertools import chain
from collections import OrderedDict
from itertools import combinations 

# Recursive

In [412]:
'''
corpus =['I1,I2,I5',
'I2,I4',
'I2,I3',
'I1,I2,I4',
'I1,I3',
'I2,I3',
'I1,I3',
'I1,I2,I3,I5',
'I1,I2,I3']

'''
corpus=['I1,I2,I5',
'I2,I3,I4',
'I3,I4',
'I1,I2,I3,I4']


vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
index_col=['T'+str(i+1)  for i in range(len(corpus))]
print(index_col)
print(X.toarray())
print(index_col)
print(X.toarray())

['i1', 'i2', 'i3', 'i4', 'i5']
['T1', 'T2', 'T3', 'T4']
[[1 1 0 0 1]
 [0 1 1 1 0]
 [0 0 1 1 0]
 [1 1 1 1 0]]
['T1', 'T2', 'T3', 'T4']
[[1 1 0 0 1]
 [0 1 1 1 0]
 [0 0 1 1 0]
 [1 1 1 1 0]]


In [413]:
columns=vectorizer.get_feature_names()
columns

['i1', 'i2', 'i3', 'i4', 'i5']

In [414]:
#L[k].apply(lambda row: row[row == 1].index, axis=1).to_list()

#L[k].columns[L[k].eq(True).any()]

In [415]:
columns
C={}
L={}
candidates=None
min_sup=2
def getMinSupport(my_table,min_sup,candidates_temp):
  if candidates_temp is None:    
    df_L=my_table.loc[:, (my_table .sum(axis=0) >= min_sup)]      #Sum of columns is greater than minimum support
    df_L=df_L[df_L.sum(axis=1) >= min_sup]                        #Sum of rows is greater than minimum support
    return df_L
  else:
    rows_singleton=my_table.index.to_list() #Get min support rows
    cols_singleton=L[1].columns.to_list()   # Get min support singleton columns
    print('rows_singleton ' ,rows_singleton)
    #print('cols_singleton ' ,cols_singleton)

    new_df=L[1][cols_singleton].copy()
    new_df=new_df[new_df.index.isin(my_table.index)]
    #print('new_df')
    #print(new_df)
    for itemset in candidates_temp:
      combination_string = ",".join(itemset)
      itemset_as_list=[]
      for item in itemset:
        itemset_as_list.append(item)
      print(itemset_as_list)
      new_df[combination_string] = new_df[itemset_as_list].all(axis=1)
    print('Before dropping singleton columns')
    print(new_df)
    new_df.drop(cols_singleton,axis=1,inplace=True)

    print('After dropping singleton columns')
    print(new_df)

    df_L=new_df.loc[:, (new_df .sum(axis=0) >= min_sup)]  #Sum of columns is greater than minimum support
    print('Sum of columns is greater than minimum support')
    print(df_L)

    if len(df_L.columns) ==1 :
      return df_L

    df_L=df_L[df_L.sum(axis=1) >= min_sup]                #Sum of rows is greater than minimum support
    print('Sum of rows is greater than minimum support')
    print(df_L)
    
  return df_L

# Python3 program to remove duplicate 
# tuples from list of tuples 

def removeDuplicates(lst): 	
	return [t for t in (set(tuple(i) for i in lst))] 
	

def getCandidateSets(arr,tuple_size,iteration):   
  temp_itemtuple_list=[]
  temp=list(combinations(arr,2))  
  for item in temp:
    mylist=(item[0]+',' +item[1]).split(',')
    t=sorted(list(dict.fromkeys(mylist))) #Order the item list lexigraphically
    if len(t) == iteration+1:
      valid_tuple=t
      temp_itemtuple_list.append(valid_tuple)

  temp_itemtuple_list=removeDuplicates(temp_itemtuple_list)   #Remove duplicate tuples from the list 
  return temp_itemtuple_list 

In [416]:
k=1
C[k]=pd.DataFrame(data=X.toarray(), index=index_col ,columns=columns)
C[k]

,i1,i2,i3,i4,i5
T1,1,1,0,0,1
T2,0,1,1,1,0
T3,0,0,1,1,0
T4,1,1,1,1,0


In [417]:
L[k]=getMinSupport(C[k],min_sup,candidates)
L[k]

,i1,i2,i3,i4
T1,1,1,0,0
T2,0,1,1,1
T3,0,0,1,1
T4,1,1,1,1


In [418]:
candidates=getCandidateSets(L[k].columns.to_list(),2,k)
candidates
k=k+1
L[k]=getMinSupport(L[k-1],min_sup,candidates)
L[k]

rows_singleton  ['T1', 'T2', 'T3', 'T4']
['i1', 'i3']
['i3', 'i4']
['i1', 'i2']
['i2', 'i4']
['i2', 'i3']
['i1', 'i4']
Before dropping singleton columns
    i1  i2  i3  i4  i1,i3  i3,i4  i1,i2  i2,i4  i2,i3  i1,i4
T1   1   1   0   0  False  False   True  False  False  False
T2   0   1   1   1  False   True  False   True   True  False
T3   0   0   1   1  False   True  False  False  False  False
T4   1   1   1   1   True   True   True   True   True   True
After dropping singleton columns
    i1,i3  i3,i4  i1,i2  i2,i4  i2,i3  i1,i4
T1  False  False   True  False  False  False
T2  False   True  False   True   True  False
T3  False   True  False  False  False  False
T4   True   True   True   True   True   True
Sum of columns is greater than minimum support
    i3,i4  i1,i2  i2,i4  i2,i3
T1  False   True  False  False
T2   True  False   True   True
T3   True  False  False  False
T4   True   True   True   True
Sum of rows is greater than minimum support
    i3,i4  i1,i2  i2,i4  i2,i3
T2   Tr

,"i3,i4","i1,i2","i2,i4","i2,i3"
T2,True,False,True,True
T4,True,True,True,True


In [419]:
candidates=getCandidateSets(L[k].columns.to_list(),2,k)
candidates
k=k+1
L[k]=getMinSupport(L[k-1],min_sup,candidates)
L[k]

rows_singleton  ['T2', 'T4']
['i2', 'i3', 'i4']
['i1', 'i2', 'i4']
['i1', 'i2', 'i3']
Before dropping singleton columns
    i1  i2  i3  i4  i2,i3,i4  i1,i2,i4  i1,i2,i3
T2   0   1   1   1      True     False     False
T4   1   1   1   1      True      True      True
After dropping singleton columns
    i2,i3,i4  i1,i2,i4  i1,i2,i3
T2      True     False     False
T4      True      True      True
Sum of columns is greater than minimum support
    i2,i3,i4
T2      True
T4      True


,"i2,i3,i4"
T2,True
T4,True


In [420]:
candidates=getCandidateSets(L[k].columns.to_list(),2,k)
candidates
k=k+1
L[k]=getMinSupport(L[k-1],min_sup,candidates)
L[k]

rows_singleton  ['T2', 'T4']
Before dropping singleton columns
    i1  i2  i3  i4
T2   0   1   1   1
T4   1   1   1   1
After dropping singleton columns
Empty DataFrame
Columns: []
Index: [T2, T4]
Sum of columns is greater than minimum support
Empty DataFrame
Columns: []
Index: [T2, T4]
Sum of rows is greater than minimum support
Empty DataFrame
Columns: []
Index: []


""


In [421]:
if (len(L[k].index) ==0):
  print('Terminated  generaion of itemsets')

Terminated  generaion of itemsets


In [422]:
L[k-1]

,"i2,i3,i4"
T2,True
T4,True


**Rule generation**

Get proper subsets 

In [423]:
from itertools import chain, combinations

def powerset(arr):
    itemset4rules_dict={}
    tuple_size=len(arr)
    for i in range(1,tuple_size+1):      
      itemset4rules_dict[i]= list(combinations(arr,i))
    return itemset4rules_dict

In [424]:
L[k-1].columns.to_list()
for col in L[k-1].columns.to_list():
  lst=col.split(',')
  itemset4rules_dict=powerset(lst)

Find support for each itemset

In [425]:
def getSupport(list_itemsets,itemset_size):
  #itemset_size is k
  s=L[itemset_size][[c for c in L[itemset_size].columns if c in list_itemsets]].reset_index().melt('index')
  temp=pd.crosstab(index=s['index'],columns=s.variable,values=s.value,aggfunc='sum',margins=True)
  return temp.iloc[-1][:-1].to_list()

In [426]:
support_dictionary={}
for key, value in itemset4rules_dict.items():
    #print(key, ' : ', value)
    #print('Itemset count :',key)
    list_itemsets=[]
    for itemset in value:      
      my_string = ','.join(str(x) for x in itemset)
      #print(my_string) 
      list_itemsets.append(my_string)

    #print(list_itemsets)
    
    list_supports=getSupport(list_itemsets,key)

    #support_dictionary = dict(zip(list_itemsets, list_supports))
    support_dictionary.update(dict(zip(list_itemsets, list_supports)))
print(support_dictionary)    

{'i2': 3, 'i3': 3, 'i4': 3, 'i2,i3': 2, 'i2,i4': 2, 'i3,i4': 2, 'i2,i3,i4': 2}


Find the items with required confidence

In [427]:
#Confidence = support {I1, I2, I3} / support {I1, I2}
#{1,3} –> ({1,3,5} – {1,3}) means 1 & 3 –> 5
#{1} –> ({1,3,5} – {1}) means 1 –> 3 & 5

In [428]:
def printRules(valid_rules):

  for lhs in valid_rules:  
    
    rhs=list(set(superset.split(','))- set(lhs.split(',')))
    
    rhs = " & ".join(rhs)

    print(lhs + '   ==>    ' + rhs)

In [429]:
superset=list(support_dictionary.keys())[-1]

superset_support=support_dictionary[list(support_dictionary.keys())[-1]] #Last item in dictionary is the superset

min_conf=50.0
valid_rules=[]

#print('Itemset','Confidence')
for key, value in support_dictionary.items():  
  subset_support=value
  confidence=(superset_support/subset_support)*100.0
  if confidence >= min_conf:
    #print(key,confidence)
    valid_rules.append(key)

print("Rules generated with  min_sup = %s and  min_conf = %s" % (min_sup, min_conf))

printRules(valid_rules[:-1])

Rules generated with  min_sup = 2 and  min_conf = 50.0
i2   ==>    i4 & i3
i3   ==>    i2 & i4
i4   ==>    i2 & i3
i2,i3   ==>    i4
i2,i4   ==>    i3
i3,i4   ==>    i2
